### Source

https://nike.ent.box.com/file/1355345336690?s=ppbfr7j7rgcftqpjjcbmav9o09opj38e

In [ ]:
# Databricks notebook source
# MAGIC %run /Users/Shrey.Bavisi@nike.com/DMA_SharedRepository/AudienceSizing_EO/#eo_functions

# COMMAND ----------

# MAGIC %md
# MAGIC Target Audience: Email Contactable

# COMMAND ----------

aud_df = spark.sql('''

SELECT DISTINCT mh.source_id AS upm_id
FROM
  (SELECT source_id, member_id
   FROM member.member_hub
   WHERE 1=1
     AND lower(dpa_status) IN ('activate','reactivate')
     AND ctry_2_cd IN ('US')
     AND lower(valid_email_format_flag) = 'true') mh

INNER JOIN comms_atc.combined_starting_universe su ON su.upm_id = mh.source_id

WHERE 1=1
  AND su.email_contactable = 1

''')


# COMMAND ----------

# estimated count for the audience size
aud_df.count()

# COMMAND ----------

aud_eo_df = spark.sql('''

SELECT DISTINCT mh.source_id AS upm_id
FROM
  (SELECT source_id, member_id
   FROM member.member_hub
   WHERE 1=1
     AND lower(dpa_status) IN ('activate','reactivate')
     AND ctry_2_cd IN ('US')
     AND lower(valid_email_format_flag) = 'true') mh

INNER JOIN comms_atc.combined_starting_universe su ON su.upm_id = mh.source_id

WHERE 1=1
  AND su.email_contactable = 1
''')

eo_metric_df = spark.sql('''
  SELECT DISTINCT gck.upm_id
  , CASE WHEN dem.7_day_demand > 0 THEN 1 else 0 END as 7_day_buyer_flag
  , dem.7_day_demand
  , dem.7_day_units
  , dem.7_day_orders
  FROM gck_common.agg_user_first_and_last_touchpoint gck

  INNER JOIN member.member_hub mh
  ON gck.upm_id = mh.source_id

  LEFT JOIN
    (SELECT DISTINCT upm_id
    , SUM(CASE WHEN order_dt >= date_sub(to_date(current_date()), 7) THEN grd_amt_excl_tax_usd END) as 7_day_demand
    , SUM(CASE WHEN order_dt >= date_sub(to_date(current_date()), 7) THEN origl_ordered_qty END) as 7_day_units
    , COUNT(DISTINCT CASE WHEN order_dt >= date_sub(to_date(current_date()), 7) THEN order_hdr_key END) as 7_day_orders
   FROM aud_select_workspace.digital_order_line_snapshot
   WHERE order_dt >= date_sub(to_date(current_date()), 7)
    AND rec_excl_ind = 0
    AND upm_id IS NOT NULL 
   GROUP BY upm_id
     ) dem 
  ON gck.upm_id = dem.upm_id
  ''')

aud_eo_metric_df = aud_eo_df.join(eo_metric_df, on = 'upm_id', how = 'left')

# COMMAND ----------

import pyspark.sql.functions as f
audience_size = aud_eo_metric_df.cache().count()

nbr_buyers = aud_eo_metric_df.filter(f.col("7_day_buyer_flag") == 1).count()

conversion_rate = nbr_buyers / audience_size

total_demand = aud_eo_metric_df.agg(f.sum("7_day_demand")).collect()[0][0]

nbr_orders = aud_eo_metric_df.agg(f.sum("7_day_orders")).collect()[0][0]

aov = total_demand / nbr_orders

avg_demand = aud_eo_metric_df.agg(f.mean("7_day_demand")).collect()[0][0]
std_demand = aud_eo_metric_df.agg(f.stddev("7_day_demand")).collect()[0][0]

# COMMAND ----------

import pandas as pd
from itertools import product
# store the notebook name to include later on in the graph
notebook_name = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get().split('/')[-1]

# assumed scaling frequency. This will be used to compute the graph. 
assumed_scaling_freq = 4

# presumed open rate of email notification - current baseline is 30% open rate
open_rate = 0.30

# list of values representing possible lift percentages. These are just common values we have seen. Add more/less depending on the use case.
lift_percentage = [0.0, 0.005, 0.01, 0.015, 0.02, 0.025, 0.03, 0.035, 0.04, 0.045, 0.05] 

# list of scaling frequencies. These are common scaling multipliers, yearly, twice yearly, quarterly, ever other month, etc. 
scaling_freq = [1, 2, 4, 6, 8, 10, 12, 17, 26, 52]

# calculation to compute the EO values. Computes for each combination of lift percentage and scaling frequency. 
eo_vals = [round(audience_size * open_rate * conversion_rate * aov * perc * freq,0) for perc, freq in product(lift_percentage,scaling_freq)]

#
combi_vals = list(product(lift_percentage,scaling_freq))
# a pandas dataframe for all the combinations of lift percentage and scaling frequency
eo_pd_df = pd.DataFrame(combi_vals, columns =['lift_perc', 'scaling_freq'])
# add a column for the EO values
eo_pd_df['EO'] = eo_vals

print(f'''
Test Name: {notebook_name}
Audience Size: {audience_size}
Number of Buyers: {nbr_buyers}
Conversion Rate: {conversion_rate:.1%}
Total Demand: $ {total_demand:.0f}
Number of Orders: {nbr_orders}
AOV: $ {aov:.0f}
Scaling Frequency: {assumed_scaling_freq}
avg_demand: {avg_demand:.2f}
std_demand: {std_demand:.2f}
''')

# COMMAND ----------

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import pandas as pd
from datetime import date

today = date.today()

plt.style.use('fivethirtyeight')   
fig, ax = plt.subplots(figsize=(8, 6))
palette = sns.color_palette("bright", 4)
g = sns.lineplot(data=eo_pd_df[eo_pd_df['scaling_freq'] == assumed_scaling_freq], x="lift_perc", y="EO")

font_color = '#525252'
csfont = {'fontname':'Georgia'}
hfont = {'fontname':'Georgia'}

# title of the notebook that the numbers were calculated in
title = f'{notebook_name}'
# add the title to the plot
fig.suptitle(title, y=.97, fontsize=22, color=font_color, **csfont)
# subtitle with some notes about when the numbers were calculated 
subtitle = f'Note: Audience criteria as of {today} using a 7 day measurement window'
# add the subtitle to the plot
plt.title(subtitle, fontsize=10, pad=10, color=font_color, **hfont)
plt.subplots_adjust(top=0.85)

# legend text that includes the values in our calculations
legend_txt = f'''
Audience Size: {human_format(audience_size)}
Baseline Conversion: {conversion_rate:.1%}
AOV: ${human_format(aov)}
Scaling Frequency: {assumed_scaling_freq}
'''
# formats the legend box
props = dict(boxstyle='round', facecolor='wheat', alpha=0.4)
# creates the legend box
ax.text(0.05, 0.98, legend_txt, transform=ax.transAxes, fontsize=10,
        verticalalignment='top', bbox=props)
# rename the x and y axises 
ax.set(xlabel='Lift Assumption', ylabel='Expected Outcome')
# reformat the y axis to be displayed in dollars
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos:  "$" + human_format(x)))
# reformat the x axis to be displayed as a percentage 
ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.1%}'.format(x)))
display(g)

# COMMAND ----------

# Pivoted dataframe displaying lift percentages and scaling frequencies. Tabular view. 
eo_pd_df.pivot(index='lift_perc', columns='scaling_freq', values='EO')

# COMMAND ----------

from statsmodels.stats.power import tt_ind_solve_power
# ratio between test and control groups
ratio = 1
# probability that the test correctly rejects the Null Hypothesis if the Alternative Hypothesis is true
power = 0.8
# significance level
alpha = 0.1
#two-sided or one-sided test
alt = 'two-sided'
# iterate through various lift percentages to calculate how much total sample is required for each lift percentage
eo_sample_size_list = [sample_solver(avg_demand, std_demand, lift, alpha, power, ratio, alt) for lift in lift_percentage[1:]]
# create a pandas dataframe to store the values
eo_sample_size_pd = pd.DataFrame(lift_percentage[1:], columns = ['lift_perc'])
# create a new column with our sample sizes
eo_sample_size_pd['sample_size'] = eo_sample_size_list

eo_sample_size_pd

# COMMAND ----------

# MAGIC %md
# MAGIC Pod 1 (Demand) Example comment
# MAGIC
# MAGIC - Estimated audience size is 6.6M consumers
# MAGIC - Baselines conversion: 1.7%
# MAGIC - Expected Lift: 0.5%
# MAGIC - Expected Treatment Viewing Rate: 5%
# MAGIC - Scaled: Monthly (12 sends)
# MAGIC - EO: $995K

# COMMAND ----------

